In [1]:
from tqdm import tqdm

from alexlib.times import Timer

from utils.elt_config import (
    DB_PATH,
    DataDirectory,
    create_all_schemas,
    get_all_schemas,
    get_cnxn,
    get_info_schema_df,
)

In [2]:
cnxn = get_cnxn(database=DB_PATH)
create_all_schemas(cnxn)

In [3]:
data_dir = DataDirectory()
data_dir.queries.load_landing_data(cnxn)

In [7]:
info_schema_df = get_info_schema_df(cnxn)
schemas = set(get_all_schemas(info_schema_df))
if "main" not in schemas:
    schemas.add("main")
if "agg" not in schemas:
    schemas.add("agg")
print("schemas:", schemas)
print("columns:", info_schema_df.columns)
info_schema_df.loc[:, ["schema", "name", "column_names"]].head(20)

schemas: {'agg', 'landing', 'main'}
columns: Index(['database', 'schema', 'name', 'column_names', 'column_types',
       'temporary'],
      dtype='object')


,schema,name,column_names
0,landing,assessments,"[assessment_id, code_module, code_presentation..."
1,landing,courses,"[code_module, code_presentation, module_presen..."
2,landing,student_assessment,"[student_id, assessment_id, date_submitted, is..."
3,landing,student_info,"[student_id, code_module, code_presentation, i..."
4,landing,student_registration,"[code_module, code_presentation, student_id, d..."
5,landing,student_vle,"[site_id, student_id, code_module, code_presen..."
6,landing,vle,"[site_id, code_module, code_presentation, acti..."
7,main,activity_type,"[id, activity_type]"
8,main,age_band,"[id, age_band]"


In [6]:
queries = data_dir.queries
timer = Timer()
for sql_file in tqdm(
    sorted(
        [
            x
            for x in queries.allchildfiles
            if x.path.suffix == ".sql" and "00_landing" not in x.path.parts
        ],
        key=lambda x: str(x.path.parent),
    )
):
    sql = sql_file.path.read_text()
    upper = sql.upper()
    print("starting:", sql_file.path.parts[-3:], end="\r")
    for cmd in ["CREATE", "DROP", "INTO"]:
        if cmd in upper:
            raise ValueError(f"{cmd} in SQL {sql_file.path}")
    stem_parts = sql_file.path.stem.split("_")
    if len(stem_parts) < 2:
        raise ValueError(f"stem {sql_file.path.stem} has less than 2 parts")
    schema = stem_parts[0]
    if schema not in schemas:
        raise ValueError(f"schema {schema} not in schemas {schemas}")
    table = "_".join(stem_parts[1:])
    obj_type = "VIEW" if table.startswith("v_") else "TABLE"
    create_sql = f"CREATE {obj_type} IF NOT EXISTS {schema}.{table} AS {sql}"
    try:
        cnxn.execute(create_sql)
    except Exception as e:
        raise e from e
    timer.log_from_start()

  2%|▏         | 2/89 [00:00<00:04, 19.15it/s]


0.0 nsng: ('01_main', '00_dimensions', 'main_age_band.sql')
0.118 sec ('01_main', '00_dimensions', 'main_activity_type.sql')


BinderException: Binder Error: Referenced column "presentation_code" not found in FROM clause!
Candidate bindings: "courses.code_presentation", "courses.module_presentation_length"
LINE 7: WHERE presentation_code IS NOT NULL
              ^